<a href="https://colab.research.google.com/github/AliTarek-1/Project-4.-Fake-News-Prediction-using-Machine-Learning-with-Python-/blob/main/Project_4_Fake_news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import re #to serach for smth in the text
from nltk.corpus import stopwords  #to remove unnecessary words in the text as the a ....
from nltk.stem.porter import PorterStemmer # the stem to remove from the words the suffix and prefix and get the root only
from sklearn.feature_extraction.text import TfidfVectorizer



In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
print(stopwords.words('english')) #these are the words that we will remove from our sentences because they dont add much meaning or value to our process

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:
fake_news_dataset=pd.read_csv('/content/train.csv')

In [ ]:

fake_news_dataset.head()


,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
fake_news_dataset.shape

(20800, 5)

In [ ]:
fake_news_dataset.describe()

,id,label
count,20800.000000,20800.000000
mean,10399.500000,0.500625
std,6004.587135,0.500012
min,0.000000,0.000000
25%,5199.750000,0.000000
50%,10399.500000,1.000000
75%,15599.250000,1.000000
max,20799.000000,1.000000


In [ ]:
fake_news_dataset.groupby('label').mean

<bound method GroupBy.mean of <pandas.core.groupby.generic.DataFrameGroupBy object at 0x7baa419a0670>>

In [ ]:
X=fake_news_dataset.drop(['label'],axis=1)
Y=fake_news_dataset['label']

In [ ]:
print(X)
print(Y)

          id  ...                                               text
0          0  ...  House Dem Aide: We Didn’t Even See Comey’s Let...
1          1  ...  Ever get the feeling your life circles the rou...
2          2  ...  Why the Truth Might Get You Fired October 29, ...
3          3  ...  Videos 15 Civilians Killed In Single US Airstr...
4          4  ...  Print \nAn Iranian woman has been sentenced to...
...      ...  ...                                                ...
20795  20795  ...  Rapper T. I. unloaded on black celebrities who...
20796  20796  ...  When the Green Bay Packers lost to the Washing...
20797  20797  ...  The Macy’s of today grew from the union of sev...
20798  20798  ...  NATO, Russia To Hold Parallel Exercises In Bal...
20799  20799  ...    David Swanson is an author, activist, journa...

[20800 rows x 4 columns]
0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 2080

In [ ]:
X_train , X_test , Y_train , Y_test= train_test_split(X,Y,train_size=0.2, stratify=Y, random_state=2)

In [ ]:
fake_news_dataset.isnull().sum()

,0
id,0
title,558
author,1957
text,39
label,0


In [ ]:
fake_news_dataset=fake_news_dataset.fillna('')

In [ ]:
fake_news_dataset['content']=fake_news_dataset['author']+' '+fake_news_dataset['title']

In [ ]:
print(fake_news_dataset['content'])

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


In [ ]:

port_stem=PorterStemmer()

In [ ]:
def stemming(content):
  stemmed_content = re.sub('[˄a-zA-Z]' , ' ' , content) #searching for regular expression and remove anything that isnot the same as regular expression and to replace it with the alphabet i mentioned
  stemmed_content=stemmed_content.lower()
  stemmed_content=stemmed_content.split()
  stemmed_content=[port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content=' '.join(stemmed_content)
  return stemmed_content

In [ ]:
fake_news_dataset['content']=fake_news_dataset['content'].apply(stemming)

In [ ]:
print(fake_news_dataset['content'])

0              : ’ ’
1            . : , -
2                  .
3                 15
4                   
            ...     
20795       . .: ’ ’
20796    . . . : , -
20797        . ’ ’ -
20798              ,
20799            -35
Name: content, Length: 20800, dtype: object


In [ ]:
vectorizer=TfidfVectorizer()
vectorizer.fit(X)

X=vectorizer.transform(X)

In [ ]:
print(X)

  (0, 1)	1.0
  (1, 3)	1.0
  (2, 0)	1.0
  (3, 2)	1.0


In [ ]:
model=LogisticRegression()


In [ ]:
model.fit(X_train,Y_train)

ValueError: could not convert string to float: "Is too much 'screen time' hurting your kids' mental growth?"

In [ ]:
training_predicition=model.predict(X_train)
training_data_accuracy=accuracy_score(training_predicition,Y_train)
print(training_data_accuracy)

ValueError: could not convert string to float: "Is too much 'screen time' hurting your kids' mental growth?"

In [ ]:
test_predicition=model.predict(X_test)
test_data_accuracy=accuracy_score(training_predicition,Y_test)
print(test _data_accuracy)

SyntaxError: invalid syntax. Perhaps you forgot a comma? (<ipython-input-72-f8ceeaa14c6a>, line 3)

In [ ]:
X_new=X_test[0]

prediction=model.predict(X_new)
print(prediction)

if(prediction[0]==0):
  print('The news is Real')
else:
  print('The news is Fake')

SyntaxError: incomplete input (<ipython-input-73-851aff39bbde>, line 8)